<a href="https://colab.research.google.com/github/yellayujin/studying/blob/main/240221_ML4(%EB%B9%85%EB%B6%84%EA%B8%B0_%EC%97%B0%EC%8A%B5).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 문제 1.
- 다음은 E-Commerce Shipping 데이터 세트이다. 주어진 훈련 데이터 세트를 활용하여 고객이 주문한 물품의 정시 도착 여부를 예측하고 해당 예측 결과를 csv 파일로 제출하시오.
- 결과 예측 양식 : 제출한 예측값의 ROC_AUC 점수 결과를 통해 영역별 배점에 따라 최종 점수가 반영될 예정

In [ ]:
import pandas as pd
pd.DataFrame({
    "ID"  : [0, 1, 2],
    "y_pred" : [0.2543, 0.1324, 0.5892]
})

,ID,y_pred
0,0,0.2543
1,1,0.1324
2,2,0.5892


## 데이터 불러오기

In [ ]:
import pandas as pd
X_test = pd.read_csv("data/data/2회/204_x_test.csv")
X_train = pd.read_csv("data/data/2회/204_x_train.csv")
y_train = pd.read_csv("data/data/2회/204_y_train.csv")

In [ ]:
y_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6599 entries, 0 to 6598
Data columns (total 2 columns):
 #   Column               Non-Null Count  Dtype
---  ------               --------------  -----
 0   ID                   6599 non-null   int64
 1   Reached.on.Time_Y.N  6599 non-null   int64
dtypes: int64(2)
memory usage: 103.2 KB


## 데이터 정보

In [ ]:
import pandas as pd
pd.DataFrame({
    "변수" : list(X_train.columns) + ["Reached.on.Time_Y.N"],
    "설명" : [
        "고객 ID 번호",
        "창고의 블록 단위 구역(A, B, C, D, F)",
        "제품 배송 방법",
        "문의 전화 수",
        "고객의 등급(1: 가장 낮음, 5: 가장 높음)",
        "제품의 비용(달러 기준)",
        "사전 구매 수량",
        "제품의 중요도",
        "성별(F: 여성, M: 남성)",
        "할인혜택",
        "그램 단위 무게",
        "정시 도착 여부(1: 도착하지 않음, 0: 정시 도착)"
    ]
})

,변수,설명
0,ID,고객 ID 번호
1,Warehouse_block,"창고의 블록 단위 구역(A, B, C, D, F)"
2,Mode_of_Shipment,제품 배송 방법
3,Customer_care_calls,문의 전화 수
4,Customer_rating,"고객의 등급(1: 가장 낮음, 5: 가장 높음)"
5,Cost_of_the_Product,제품의 비용(달러 기준)
6,Prior_purchases,사전 구매 수량
7,Product_importance,제품의 중요도
8,Gender,"성별(F: 여성, M: 남성)"
9,Discount_offered,할인혜택


# 답안 작성란

In [ ]:
# 셀 하나에 쓰고 본 셀 돌리는 연습
# 데이터 확인
# print(X_train.head())
 # 범주형, 수치형 섞여있는데 Pipeline배우기 전에는 하나하나 따로 원핫인코딩 or 라벨인코딩 등 수행했어야

# print(y_train.head())

# 결측치 확인
# print(X_train.isnull().sum())
# print(X_test.isnull().sum())
# print(y_train.isnull().sum())

# 불필요한 컬럼 삭제
X_train1 = X_train.drop('ID', axis=1)
# X_train1 = X_train.pop('ID')로 해도 됨, 단, pop은 자체적으로 inplace 들어있다고 생각하기
X_test1 = X_test.drop('ID', axis=1)
y_train1 = y_train.drop('ID', axis=1)

# print(X_train1.columns)
# print(X_test1.columns)

# 종속변수 구성 확인
# print(y_train1.value_counts())  # 비율 차이가 39:26. stratify옵션

# 자료 분포 확인
# print(X_train1.describe())   # 표준편차를 확인->너무 큰 컬럼 있으면
 # 피처 엔지니어링 때 StandardScaler보다 RobustScaler


# Pipeline ----------------------------------------------------
# pipeline 사용시, 사용하고자 하는 컬럼명을 추출하는 것이 중요
# 숫자형 컬럼 추출
 # 근데 Customer_rating은 문자(서열)로 바꿔서 아래 문자형으로
X_train1['Customer_rating'] = X_train1['Customer_rating'].astype('category')

import numpy as np # 라이브러리는 무조건 위에 쓰는 게 좋음
numbers_df = X_train1.select_dtypes(include=np.number)  # 이 코드 익숙해지자
# print(numbers_df.head(1))

# 문자형 컬럼 추출
object_df = X_train1.select_dtypes(exclude=np.number)
# print(object_df.head(1))
 # OrdinalEncoder과 One-Hot Encoder 쓸 것 분리
 # Product_importance: 서열척도(OrdinalEncoder) , Warehouse_block: 서열주긴 좀 그럼(명목)(One-Hot Encoder)


# train, test데이터 분리---------------------------------------------------
 # (위 수치, 문자로 나눈 건 각기 다르게 전처리하기 위함)
# train으로 또 나누넹
from sklearn.model_selection import train_test_split

X_tr, X_te, y_tr, y_te = train_test_split(X_train1, y_train1, random_state=0, stratify=y_train1, test_size=0.3)
# print(X_tr.shape, X_te.shape, y_tr.shape, y_te.shape)


# 모델링----------------------------------------------------
# 시험에서는 이 부분이 가장 중요(외우세용)
# 각 부분 구조 꼭 알고있기
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder, RobustScaler, MinMaxScaler, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import roc_auc_score
from lightgbm import LGBMClassifier

# RandomizedSearchCV에 대입될 예정. 맨 위 2가지는 필수.
param_grid = {
    'classifier__n_estimators': np.arange(30, 1000, 100),  # 언더바 2개임, 이 단어는 아래 Pipeline의 단어와 일치해야
    # n_estimators: 매개변수 이름
    'classifier__max_depth': [None] + list(np.arange(5, 30, 5)), # none도 포함시킴
    'classifier__min_samples_split': [2, 5, 10], # np.arange()로 쓰는 게 무작위 수 넣는 것보다 원래는 더 좋음
    'classifier__min_samples_leaf': [1, 2, 4],
    'classifier__max_features': ['auto', 'sqrt', 'log2']
}

# ColumnTransformer
column_transformer = ColumnTransformer([
    ('scaler', RobustScaler(), numbers_df.columns), # scaler
    ('ord_encoder', OrdinalEncoder(), ['Customer_rating', 'Product_importance']), # 서열척도 처리, categories=[['low', 'medium','high']]에 중첩리스트 형태로!
    ('ohc_encoder', OneHotEncoder(), ['Warehouse_block','Mode_of_Shipment', 'Gender']), # 명목척도 처리
], remainder = 'passthrough')   # remainder='passthrough': 학습하지 않은 데이터를 테스트데이터 활용(필수, 안 하면 에러날수도)

pipeline = Pipeline([('preprocessor', column_transformer), ('classifier', RandomForestClassifier(random_state=0))])

# GridSearchCV와 코드 스타일 비슷함. Randomized가 Grid보다 나으니 이거 쓰자
random_search = RandomizedSearchCV(
    estimator = pipeline,
    random_state=0,
    param_distributions = param_grid,
    n_iter=3,
    scoring='roc_auc',
    cv = 3,
    verbose=3,
    n_jobs = -1 # CPU cores를 모두 사용하겠다는 뜻
)

# 모형학습
random_search.fit(X_tr, y_tr)

# 예측값 산출
predictions = random_search.predict_proba(X_te)

# 모형평가
roc_auc_score(y_te, predictions[:, 1])

Fitting 3 folds for each of 3 candidates, totalling 9 fits


In [ ]:
# 답안 제출 코드
final_preds = random_search.predict_proba(X_test)
result = pd.DataFrame({"ID" : X_test['ID'], "pred": final_preds[:, 1]})
result.head(1)
result.to_csv("00000.csv", index=False)

## 데이터 확인

In [ ]:
X_train.head(1)

,ID,Warehouse_block,Mode_of_Shipment,Customer_care_calls,Customer_rating,Cost_of_the_Product,Prior_purchases,Product_importance,Gender,Discount_offered,Weight_in_gms
0,9902,F,Ship,3,5,214,2,medium,F,6,4578


In [ ]:
X_test.head(1)

,ID,Warehouse_block,Mode_of_Shipment,Customer_care_calls,Customer_rating,Cost_of_the_Product,Prior_purchases,Product_importance,Gender,Discount_offered,Weight_in_gms
0,7007,C,Ship,4,1,238,3,high,F,4,4910


In [ ]:
y_train.head(1)

,ID,Reached.on.Time_Y.N
0,9902,0


## 데이터 확인

### 각 컬럼의 정보 확인

In [ ]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6599 entries, 0 to 6598
Data columns (total 11 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   ID                   6599 non-null   int64 
 1   Warehouse_block      6599 non-null   object
 2   Mode_of_Shipment     6599 non-null   object
 3   Customer_care_calls  6599 non-null   int64 
 4   Customer_rating      6599 non-null   int64 
 5   Cost_of_the_Product  6599 non-null   int64 
 6   Prior_purchases      6599 non-null   int64 
 7   Product_importance   6599 non-null   object
 8   Gender               6599 non-null   object
 9   Discount_offered     6599 non-null   int64 
 10  Weight_in_gms        6599 non-null   int64 
dtypes: int64(7), object(4)
memory usage: 567.2+ KB


In [ ]:
X_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4400 entries, 0 to 4399
Data columns (total 11 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   ID                   4400 non-null   int64 
 1   Warehouse_block      4400 non-null   object
 2   Mode_of_Shipment     4400 non-null   object
 3   Customer_care_calls  4400 non-null   int64 
 4   Customer_rating      4400 non-null   int64 
 5   Cost_of_the_Product  4400 non-null   int64 
 6   Prior_purchases      4400 non-null   int64 
 7   Product_importance   4400 non-null   object
 8   Gender               4400 non-null   object
 9   Discount_offered     4400 non-null   int64 
 10  Weight_in_gms        4400 non-null   int64 
dtypes: int64(7), object(4)
memory usage: 378.3+ KB


### 결측치 확인
- 결측치는 존재하지 않는 것 확인

In [ ]:
X_train.isnull().sum()

ID                     0
Warehouse_block        0
Mode_of_Shipment       0
Customer_care_calls    0
Customer_rating        0
Cost_of_the_Product    0
Prior_purchases        0
Product_importance     0
Gender                 0
Discount_offered       0
Weight_in_gms          0
dtype: int64

In [ ]:
X_test.isnull().sum()

ID                     0
Warehouse_block        0
Mode_of_Shipment       0
Customer_care_calls    0
Customer_rating        0
Cost_of_the_Product    0
Prior_purchases        0
Product_importance     0
Gender                 0
Discount_offered       0
Weight_in_gms          0
dtype: int64

### 컬럼 삭제

In [ ]:
X_train_id = X_train.pop('ID')
X_test_id = X_test.pop('ID')

X_train.shape, X_test.shape

((6599, 10), (4400, 10))

In [ ]:
X_train.shape, X_test.shape

((6599, 10), (4400, 10))

## 종속변수 데이터셋 확인

In [ ]:
y_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6599 entries, 0 to 6598
Data columns (total 2 columns):
 #   Column               Non-Null Count  Dtype
---  ------               --------------  -----
 0   ID                   6599 non-null   int64
 1   Reached.on.Time_Y.N  6599 non-null   int64
dtypes: int64(2)
memory usage: 103.2 KB


In [ ]:
y_train['Reached.on.Time_Y.N'].value_counts()

Reached.on.Time_Y.N
1    3938
0    2661
Name: count, dtype: int64

- 비율 차이는 크지 않지만, stratify 옵션을 넣어준다.

### 데이터 확인

In [ ]:
X_train.describe()

,Customer_care_calls,Customer_rating,Cost_of_the_Product,Prior_purchases,Discount_offered,Weight_in_gms
count,6599.000000,6599.000000,6599.000000,6599.000000,6599.000000,6599.000000
mean,4.058494,2.993181,209.587968,3.565086,13.236703,3638.925746
std,1.134313,1.415857,48.423993,1.533984,16.066606,1632.253471
min,2.000000,1.000000,96.000000,2.000000,1.000000,1001.000000
25%,3.000000,2.000000,169.000000,3.000000,4.000000,1844.000000
50%,4.000000,3.000000,213.000000,3.000000,7.000000,4152.000000
75%,5.000000,4.000000,251.000000,4.000000,10.000000,5049.500000
max,7.000000,5.000000,310.000000,10.000000,65.000000,7401.000000


- RobustScaler 사용할 것

In [ ]:
import numpy as np
object_df = X_train.select_dtypes(exclude=np.number)
object_df.head(3)

,Warehouse_block,Mode_of_Shipment,Product_importance,Gender
0,F,Ship,medium,F
1,A,Ship,medium,F
2,A,Ship,low,M


In [ ]:
numbers_df = X_train.select_dtypes(include=np.number)
numbers_df.columns

Index(['Customer_care_calls', 'Customer_rating', 'Cost_of_the_Product',
       'Prior_purchases', 'Discount_offered', 'Weight_in_gms'],
      dtype='object')

In [ ]:
object_df['Product_importance'].value_counts()

Product_importance
low       3170
medium    2854
high       575
Name: count, dtype: int64

- Product_importance must be ordinal encoding
    + low, medium, high

## 데이터셋 분리

In [ ]:
from sklearn.model_selection import train_test_split
X_tr, X_te, y_tr, y_te = train_test_split(
    X_train, y_train['Reached.on.Time_Y.N'],
    stratify=y_train['Reached.on.Time_Y.N'],
    random_state=42,
    test_size=0.25
)

X_tr.shape, X_te.shape, y_tr.shape, y_te.shape

((4949, 10), (1650, 10), (4949,), (1650,))

In [ ]:
X_tr

,Warehouse_block,Mode_of_Shipment,Customer_care_calls,Customer_rating,Cost_of_the_Product,Prior_purchases,Product_importance,Gender,Discount_offered,Weight_in_gms
2220,B,Road,6,5,275,4,low,F,6,1872
641,B,Road,5,3,267,6,low,M,10,1027
5738,F,Road,4,5,246,2,medium,M,31,2415
1072,B,Road,4,5,176,3,high,F,1,5096
4973,A,Ship,3,1,144,2,medium,M,8,5378
...,...,...,...,...,...,...,...,...,...,...
669,F,Road,5,2,164,2,medium,M,8,5717
3166,F,Ship,4,2,266,2,low,F,8,5516
213,A,Ship,6,3,273,5,low,M,8,1746
3375,C,Road,3,5,109,5,low,M,27,1581


## 모형 학습 및 예측값

In [ ]:
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder, RobustScaler, MinMaxScaler, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import roc_auc_score
from lightgbm import LGBMClassifier

param_grid = {
    'classifier__n_estimators': np.arange(30, 1000, 100),
    'classifier__max_depth': [None] + list(np.arange(5, 30, 5)),
    'classifier__min_samples_split': [2, 5, 10],
    'classifier__min_samples_leaf': [1, 2, 4],
    'classifier__max_features': ['auto', 'sqrt', 'log2'],
    # Add more parameters to tune as per your requirement
}

column_transformer = ColumnTransformer([
    ("scaler", MinMaxScaler(), numbers_df.columns),
    ("ohc_encoder", OneHotEncoder(), ['Warehouse_block', 'Mode_of_Shipment', 'Gender']),
    ("pd_imp_encoder", OrdinalEncoder(categories=[['low', 'medium', 'high']]), ['Product_importance'])
], remainder="passthrough")

pipeline = Pipeline([
    ("preprocessor", column_transformer),
    ("classifier", LGBMClassifier(random_state=42))
])

# Create the RandomizedSearchCV object
random_search = RandomizedSearchCV(
    estimator=pipeline,
    param_distributions=param_grid,
    n_iter=10,  # Number of parameter settings that are sampled
    scoring='roc_auc',  # Evaluation metric
    cv=5,  # Number of cross-validation folds
    verbose=3,
    n_jobs=-1  # Use all available CPU cores
)

# 모형학습
random_search.fit(X_tr, y_tr)

# 예측값 산출
predictions = random_search.predict_proba(X_te)

# 모형평가
roc_auc_score(y_te, predictions[:, 1])

Fitting 5 folds for each of 10 candidates, totalling 50 fits
[LightGBM] [Warning] Unknown parameter: min_samples_split
[LightGBM] [Warning] min_data_in_leaf is set with min_child_samples=20, will be overridden by min_samples_leaf=1. Current value: min_data_in_leaf=1
[LightGBM] [Warning] Unknown parameter: max_features
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Unknown parameter: min_samples_split
[LightGBM] [Warning] min_data_in_leaf is set with min_child_samples=20, will be overridden by min_samples_leaf=1. Current value: min_data_in_leaf=1
[LightGBM] [Warning] Unknown parameter: max_features
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Info] Number of positive: 2953, number of negative: 1996
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.0003

0.73232319377123

## 결과 제출

In [ ]:
final_preds = random_search.predict_proba(X_test)
result = pd.DataFrame({"ID" : X_test_id, "pred": final_preds[:, 1]})
result.head(1)

[LightGBM] [Warning] Unknown parameter: min_samples_split
[LightGBM] [Warning] min_data_in_leaf is set with min_child_samples=20, will be overridden by min_samples_leaf=1. Current value: min_data_in_leaf=1
[LightGBM] [Warning] Unknown parameter: max_features
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).


,ID,pred
0,7007,0.350926


In [ ]:
result.to_csv("00000.csv", index=False)